In [12]:
import nltk
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from evaluate import load
from nltk.tokenize import sent_tokenize
import numpy as np
import accelerate

In [13]:
metric = load('rouge')
nltk.download('punkt_tab')
dataset = load_dataset("cnn_dailymail", "3.0.0")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Gabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [14]:
def add_sentenum(example):
    sents = sent_tokenize(example['highlights'])
    num_sents = len(sents)
    prefix = f"[SN]{num_sents} [SEP]"
    enumerated = ' '.join([f'[SN]{i+1} {s}' for i, s in enumerate(sents)])
    example['summary_with_sentenum'] = f"{prefix} {enumerated}"
    return example

def tokenize_function(example):
    # Tokenize input article
    model_input = tokenizer(
        example["article"], 
        max_length=1024, 
        truncation=True, 
        padding="max_length"
    )
    
    # Tokenize sentence-enumerated target summary
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["summary_with_sentenum"], 
            max_length=256, 
            truncation=True, 
            padding="max_length"
        )
    
    model_input["labels"] = labels["input_ids"]
    return model_input

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects newline-separated sentences
    decoded_preds = ["\n".join(pred.strip().split('. ')) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split('. ')) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {key: value.mid.fmeasure for key, value in result.items()}

In [16]:
dataset_sentenum = dataset.map(add_sentenum)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
sentenum_tokens = [f"[SN]{i}" for i in range(1, 21)] + ["[SN]", "[SEP]"]
special_tokens_dict = {"additional_special_tokens": sentenum_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

22

In [18]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [19]:
model_checkpoints = "facebook/bart-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


BartScaledWordEmbedding(50287, 768, padding_idx=1)

In [20]:
tokenized_dataset = dataset_sentenum.map(tokenize_function, batched=True, remove_columns=['id','article', 'highlights'])

In [21]:
train_sample = tokenized_dataset['train'].shuffle(seed=123).select(range(5000))
validation_sample = tokenized_dataset['validation'].shuffle(seed=123).select(range(500))
test_sample = dataset_sentenum['test'].shuffle(seed=123).select(range(500))

In [22]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
max_input = 512

In [24]:
accelerator = accelerate.Accelerator(mixed_precision='fp16')
args = Seq2SeqTrainingArguments(
    '../summ', #save directory
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size= 4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    eval_accumulation_steps=2,
    no_cuda=False,
    fp16=torch.cuda.is_available()
    )

## Trening

In [14]:
trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_sample,
    eval_dataset=validation_sample,
    data_collator=collator,
    processing_class=tokenizer,
    compute_metrics=compute_rouge
)

In [15]:
trainer.train()

Step,Training Loss
500,1.711900
1000,0.516200
1500,0.474500


C:\Users\Gabi\PycharmProjects\PJN_cuda\.venv\Lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1560, training_loss=0.8843606410882412, metrics={'train_runtime': 46016.7104, 'train_samples_per_second': 1.087, 'train_steps_per_second': 0.034, 'total_flos': 3.030634092036096e+16, 'train_loss': 0.8843606410882412, 'epoch': 9.9408})

In [18]:
trainer.save_model('./model_sentenum2')

## Clean model

In [21]:
def tokenize_function(example):
    # Tokenize input article
    model_input = tokenizer(
        example["article"],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    # Tokenize sentence-enumerated target summary
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["highlights"],
            max_length=256,
            truncation=True,
            padding="max_length"
        )

    model_input["labels"] = labels["input_ids"]
    return model_input

In [23]:
model_checkpoints = "facebook/bart-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['id','article', 'highlights'])

In [ ]:
train_sample = tokenized_dataset['train'].shuffle(seed=123).select(range(5000))
validation_sample = tokenized_dataset['validation'].shuffle(seed=123).select(range(500))
test_sample = dataset['test'].shuffle(seed=123).select(range(500))

In [29]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

trainer_clean = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_sample,
    eval_dataset=validation_sample,
    data_collator=collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer_clean.train()

Step,Training Loss
500,1.625300
1000,0.487800
1500,0.454400


C:\Users\Gabi\PycharmProjects\PJN_cuda\.venv\Lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1560, training_loss=0.8402925662505321, metrics={'train_runtime': 46288.4383, 'train_samples_per_second': 1.08, 'train_steps_per_second': 0.034, 'total_flos': 3.030634092036096e+16, 'train_loss': 0.8402925662505321, 'epoch': 9.9408})

In [32]:
trainer_clean.save_model('./model_clean')

## Test

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
sentenum_tokens = [f"[SN]{i}" for i in range(1, 21)] + ["[SN]", "[SEP]"]
special_tokens_dict = {"additional_special_tokens": sentenum_tokens}
tokenizer.add_special_tokens(special_tokens_dict)


test_sample = dataset['test'].shuffle(seed=123).select(range(500))

In [29]:
model_trained = AutoModelForSeq2SeqLM.from_pretrained('../model_sentenum')
trainer_trained = Seq2SeqTrainer(
    model_trained, 
    args,
    processing_class=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
test_article = test_sample[10]['article']
max_input=512

# Tokenize
model_input = tokenizer(
    test_article,
    max_length=max_input,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
model_input = {k: v.to(trainer_trained.model.device) for k, v in model_input.items()}
# Generate
with torch.no_grad():
    generated_ids = trainer_trained.model.generate(
        input_ids=model_input['input_ids'],
        attention_mask=model_input['attention_mask'],
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

# Decode
output = tokenizer.decode(generated_ids[0])
print(output)

In [9]:
predictions = []
references = []

count = 0
for sample in test_sample:  # Your test or validation dataset
    input_text = sample["article"]
    reference_summary = sample["highlights"]

    # Tokenize and generate
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4
        )

    # Decode
    pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(reference_summary)
    
    count += 1
    print(count)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [34]:
results = metric.compute(predictions=predictions, references=references, use_stemmer=True)

In [35]:
for key in results:
    print(f"{key}: {results[key]:.4f}")


rouge1: 0.4054
rouge2: 0.1860
rougeL: 0.2837
rougeLsum: 0.3484
